https://adventofcode.com/2022/day/14

In [1]:
f = open("input.txt", "r")

s = read(f, String)

data = s

close(f)

function getData(str)
  lines = split(strip(str), "\n")
  instructions = map(line -> split(line, " -> "), lines)
  rock_instructions = []
  left_bound = 500
  right_bound = 500
  bottom_bound = 0

	# loops through the parse instructions string into pairs of coordinates
	# also finds the min x and max x, y to the define the spatial bounds of the simulation
  for instruction in instructions
    rock_instruction = []
    for coordinates in instruction
      x, y = split(coordinates, ",")
      x = parse(Int, x)
      y = parse(Int, y)
      push!(rock_instruction, [x, y])

      left_bound = min(left_bound, x)
      bottom_bound = max(bottom_bound, y)
      right_bound = max(right_bound, x)
    end
    push!(rock_instructions, rock_instruction)
  end

	# shift the instructions to 1, 1 being the top left corner
  rock_instructions = shift_coordinates(rock_instructions, -left_bound + 1,1)
	# shift the right_bound relative to 1, 1
  right_bound = right_bound - left_bound + 1
	# shift the source_coordinates relative to 1, 1
	source_coordinates = [500-left_bound+1,1]

	# defines the boundaries left, bottom, right, top respectively
  boundaries = [1, bottom_bound+1, right_bound, 1]


  return rock_instructions, boundaries, source_coordinates
end

# shifts the coordinates of the instructions array
function shift_coordinates(instructions, x, y)
  for instruction in instructions
    for coordinates in instruction
      coordinates[1] = coordinates[1] + x
      coordinates[2] = coordinates[2] + y
    end
  end

  return instructions
end

shift_coordinates (generic function with 1 method)

In [2]:
# an object in the window grid
mutable struct Object
	type::String
	char::Char
end

# the window the simulation will run in
mutable struct Window
  height::Int
  width::Int
	boundaries::Vector{Int}
  grid::Matrix{Object}
	source_coordinates::Vector{Int}
	simulation_complete::Bool
	sand_added::Int
	function Window(
		height::Int,
		width::Int,boundaries::Vector{Int},
		grid::Matrix{Object},
		source_coordinates::Vector{Int}
	)
		return new(height, width, boundaries, grid, source_coordinates, false, 0)
	end
end

# creates the window object to run the simulation
function generateWindow(data)
  rock_instructions, boundaries, source_coordinates = data
	left_bound, bottom_bound, right_bound , top_bound= boundaries
	height = bottom_bound
	width = right_bound

	grid = Matrix{Object}(undef, width, height)
	window = Window(height, width, boundaries, grid, source_coordinates)

	# fills the grid with air objects
	for x in 1:width, y in 1:height
		object = Object("air", '.')
		grid[x, y] = object
	end

	# shows the source in the grid
	s_x, s_y = source_coordinates
	grid[s_x, s_y].char = '+'

	# adds rocks based on the rock_instructions
	generate_rocks(window, rock_instructions)
	return window
end

# creates rocks in the window based on rock_instructions
function generate_rocks(window, rock_instructions)
	height = window.height
	width = window.width
	grid = window.grid

	for instruction in rock_instructions
		previous = instruction[1]
		for coordinates in instruction[2:end]
			current = coordinates
			# orders the start to be top and left most and end to be bottom and right most
			start_x = min(previous[1], current[1])
			start_y = min(previous[2], current[2])
			end_x = max(previous[1], current[1])
			end_y = max(previous[2], current[2])

			# adds rocks left -> right and top -> bottom from start to end
			for x = start_x:end_x, y = start_y:end_y
				grid[x, y].type = "rock"
				grid[x, y].char = '#'
			end

			previous = current
		end
	end

	return window
end

# prints a representation of the simulation in the console
function draw_window(window::Window)
	height = window.height
	width = window.width
	grid = window.grid

	line = ""
	for y = 1:height
		for x  = 1:width
			line *=  grid[x, y].char
		end
		println(line)
		line = ""
	end
end

# adds sand in the window until a grain falls out of bounds or blocks the source
function add_sand(window::Window)
	sand_coordinates = window.source_coordinates

	while !window.simulation_complete
		drop_sand(window, sand_coordinates)
	end

	return count
end

function drop_sand(window::Window, sand_coordinates)
	x, y = sand_coordinates
	grid = window.grid

	left_bound = window.boundaries[1]
	right_bound = window.boundaries[3]

	y-=1

	# sand keeps falling in air until it meets rock or sand:("not air")
	while grid[x, y+1].type == "air"
		y += 1
		
		# if the there is air below keep falling (y +=1)
		grid[x, y+1].type === "air" && continue

		# if sliding to the left puts it out of bounds, simulation is complete
		if x-1 < left_bound
			window.simulation_complete = true
			return
		end

		# if the lower left is air fall there
		if grid[x-1, y+1].type === "air"
			x -= 1
			continue
		end

		# if sliding to the right puts it out of bounds, simulation is complete
		if	x+1 > right_bound
			window.simulation_complete = true
			return
		end
		
		# if the lower right is air fall there
		if grid[x+1, y+1].type === "air"
			x += 1
			continue
		end
		
	end

	# set the object in the grid to sand
	grid[x, y].type = "sand"
	grid[x, y].char = 'o'

	# add to the sand added count
	window.sand_added += 1

	# if the sand blocks the source_coordinates the simulation is complete
	if	x == sand_coordinates[1] && y == sand_coordinates[2]
		window.simulation_complete = true
	end

	return window
end

# gets the number of sand added in the simulation
function getSandCount(data)
	data = getData(data)
	window = generateWindow(data)
	add_sand(window)
	# draw_window(window)
	return window.sand_added
end

getSandCount(data)

755

In [3]:
# modifies the bounds of the data and adds a line of rock along the bottom
function processData(data)
	rock_instructions, boundaries, source_coordinates = data

	left_bound, bottom_bound, right_bound , top_bound= boundaries

	# increases the bottom bound by 2
	bottom_bound += 2

	# calculates the amount to shift the coordinates to the left to guarantee the sand can reach the source
	left_shift = bottom_bound-source_coordinates[1]
	source_coordinates[1] += left_shift

	# the sand is guaranteed to reach the source if the right is twice the height from the rock line
	right_bound = bottom_bound*2

	boundaries = [left_bound, bottom_bound, right_bound , top_bound]

	# shifts the instructions by left_shift
	rock_instructions = shift_coordinates(rock_instructions,left_shift,0)

	# adds rock instructions to create the rock line at the bottom of the grid
	push!(rock_instructions, [[1, bottom_bound],[right_bound, bottom_bound]])

	return rock_instructions, boundaries, source_coordinates
end

function getSandCount2(data)
	window = data |> getData |> processData |> generateWindow
	add_sand(window)
	# draw_window(window)
	return window.sand_added
end


getSandCount2(data)

29805